# RNN Implementation using NumPy

- Sequence input, Char-level, Batch training, Python 3

- You can train RNNs using different vector representations. (Try "glove" or "word2vec" to train the sequence of words instead of the "One-hot-vector")

- This is a slightly modified version of the original code(Karpathy).

URL

-> https://github.com/JY-Yoon

-> Original code(Karpathy) : https://gist.github.com/karpathy/d4dee566867f8291f086



In [1]:
import numpy as np

In [2]:
# load text data

txt_data = "abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz abcdefghijklmnopqrstuvwxyz " # input data
# txt_data = open('input.txt', 'r').read() # test external files

chars = list(set(txt_data)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(txt_data)

print("unique characters : ", num_chars) # You can see the number of unique characters in your input data.
print("txt_data_size : ", txt_data_size)

unique characters :  27
txt_data_size :  81


# One hot encoding

In [3]:
# one hot encode
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" retruns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in txt_data] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'c': 0, 's': 1, 'd': 2, 'q': 3, 'p': 4, ' ': 5, 'x': 6, 'h': 7, 'r': 8, 'k': 9, 'a': 10, 'g': 11, 'n': 12, 'u': 13, 'i': 14, 'o': 15, 'f': 16, 'v': 17, 'b': 18, 'y': 19, 'w': 20, 'z': 21, 't': 22, 'j': 23, 'e': 24, 'm': 25, 'l': 26}
----------------------------------------------------
{0: 'c', 1: 's', 2: 'd', 3: 'q', 4: 'p', 5: ' ', 6: 'x', 7: 'h', 8: 'r', 9: 'k', 10: 'a', 11: 'g', 12: 'n', 13: 'u', 14: 'i', 15: 'o', 16: 'f', 17: 'v', 18: 'b', 19: 'y', 20: 'w', 21: 'z', 22: 't', 23: 'j', 24: 'e', 25: 'm', 26: 'l'}
----------------------------------------------------
[10, 18, 0, 2, 24, 16, 11, 7, 14, 23, 9, 26, 25, 12, 15, 4, 3, 8, 1, 22, 13, 17, 20, 6, 19, 21, 5, 10, 18, 0, 2, 24, 16, 11, 7, 14, 23, 9, 26, 25, 12, 15, 4, 3, 8, 1, 22, 13, 17, 20, 6, 19, 21, 5, 10, 18, 0, 2, 24, 16, 11, 7, 14, 23, 9, 26, 25, 12, 15, 4, 3, 8, 1, 22, 13, 17, 20, 6, 19, 21, 5]
----------------------------------------------------
data length :  81


In [4]:
# Not actually used.

onehot_encoded = []

for ix in integer_encoded: # ix is an index mapped to a unique character.
    letter = [0 for _ in range(len(chars))] # A list len is equal to the number of unique characters and whose elements are all zero.
    letter[ix] = 1 # 'letter' is a one-hot vector.
    onehot_encoded.append(letter) # Add a 1d list(a vector for one character).
onehot_encoded = np.array(onehot_encoded) # list to np-array   

print(onehot_encoded.shape)     #  = (len(data),len(chars))
print(onehot_encoded)

# invert encoding
inverted = int_to_char[np.argmax(onehot_encoded[0])] # "argmax" returns the index of the largest value. 
print(inverted)


(81, 27)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
a


# Initialization

In [5]:
# hyperparameters

iteration = 5000
sequence_length = 10
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
hidden_size = 100  # size of hidden layer of neurons.  
learning_rate = 1e-1


# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

# Forward propagation

In [6]:
def forwardprop(inputs, targets, h_prev):
        
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
 
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs 

# Backward propagation

In [7]:
def backprop(ps, inputs, hs, xs):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

# Training

In [8]:
data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 


for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in txt_data[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in txt_data[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(txt_data) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
    
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer
        
    if i % 100 == 0:
        print ('iter %d, loss: %f' % (i, loss)) # print progress



iter 0, loss: 2.383809
iter 100, loss: 0.013654
iter 200, loss: 0.005670
iter 300, loss: 0.003543
iter 400, loss: 0.002533
iter 500, loss: 0.001952
iter 600, loss: 0.001580
iter 700, loss: 0.001318
iter 800, loss: 0.001125
iter 900, loss: 0.000980
iter 1000, loss: 0.000866
iter 1100, loss: 0.000774
iter 1200, loss: 0.000699
iter 1300, loss: 0.000636
iter 1400, loss: 0.000584
iter 1500, loss: 0.000541
iter 1600, loss: 0.000503
iter 1700, loss: 0.000470
iter 1800, loss: 0.000441
iter 1900, loss: 0.000415
iter 2000, loss: 0.000392
iter 2100, loss: 0.000371
iter 2200, loss: 0.000352
iter 2300, loss: 0.000335
iter 2400, loss: 0.000319
iter 2500, loss: 0.000305
iter 2600, loss: 0.000291
iter 2700, loss: 0.000279
iter 2800, loss: 0.000268
iter 2900, loss: 0.000258
iter 3000, loss: 0.000248
iter 3100, loss: 0.000239
iter 3200, loss: 0.000231
iter 3300, loss: 0.000223
iter 3400, loss: 0.000216
iter 3500, loss: 0.000209
iter 3600, loss: 0.000203
iter 3700, loss: 0.000197
iter 3800, loss: 0.00019

# Prediction

In [9]:
def predict(test_char, length):
    x = np.zeros((num_chars, 1)) 
    x[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    for t in range(length):
        h = np.tanh(np.dot(W_xh, x) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        x = np.zeros((num_chars, 1)) # init
        x[ix] = 1 
        ixes.append(ix) # list
    txt = ''.join(int_to_char[i] for i in ixes)
    print ('----\n %s \n----' % (txt, ))

In [10]:
predict('a',30) # (char, len of output)

----
 bcdefghijklmnopqrstuvwxyz abcd 
----


In [11]:
predict('b',30)

----
 cdefghijklmnopqrstuvwxyz abcde 
----
